In [ ]:
import numpy as np
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from skimage.color import rgba2rgb, gray2rgb,rgb2hsv, rgb2gray
from skimage.feature import hog
from skimage import color
from skimage.feature import graycomatrix, graycoprops


# PREPROCESADO DE IMÁGENES:

In [ ]:
def aplanar_imagen(imagen):

    # Convertir la imagen a un array de NumPy
    array_imagen = np.array(imagen)
    
    # Asegurarse de que la imagen esté en escala de grises o RGB
    if len(array_imagen.shape) == 3:  # Imagen RGB
        # Aplana la imagen manteniendo todos los canales
        vector_aplanado = array_imagen.flatten()
    else:  # Imagen en escala de grises
        vector_aplanado = array_imagen.flatten()
    
    return vector_aplanado

def dimension_imagenes(carpeta_imgs,carpeta_masks,target_size=(280,280)):

# Tamaño deseado
  # Reemplaza con el tamaño deseado, por ejemplo, (256, 256)

# Lista de imágenes (rutas o imágenes cargadas)

    resized_images = []
    resized_masks = []
    for imagen in carpeta_imgs:
        with Image.open(imagen) as img:
            resized_img = img.resize(target_size)  # Redimensiona la imagen
            resized_img.convert('RGB')
            resized_images.append(resized_img)  # Guarda la imagen redimensionada
    
    for mask in carpeta_masks:
        with Image.open(mask) as msk:
            resized_mask = msk.resize(target_size)  # Redimensiona la imagen
            resized_mask.convert('L')

            resized_masks.append(resized_mask)

    return resized_images, resized_masks

def aplicar_mascara(image,mascara):
    masked_image = Image.composite(image, Image.new("RGBA", image.size, (0, 0, 0, 0)), mascara)
    return masked_image

# IMPORTAMOS LAS IMÁGENES

In [ ]:

image_folder = 'objects/images/flamingo'
mask_folder = 'objects/masks/flamingo'

images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg'))]
masks = [os.path.join(mask_folder, f) for f in os.listdir(mask_folder) if f.endswith(('.png', '.jpg'))]
vectores_aplanados = [aplanar_imagen(imagen) for imagen in images]
vectores_aplanados = [aplanar_imagen(mask) for mask in masks]

resized_images,resized_masks=dimension_imagenes(images,masks)

flamencazos=[]
for i in range(len(resized_images)):
    imagen_mascara=aplicar_mascara(resized_images[i],resized_masks[i])
    imagen_mascara.convert('RGB')
    img_prueba=np.array(imagen_mascara)
    flamencazos.append(img_prueba)
et_flam=[]
for i in range(len(flamencazos)):
    et_flam.append('O')


image_folder = 'objects/images/elephant'
mask_folder = 'objects/masks/elephant'

images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg'))]
masks = [os.path.join(mask_folder, f) for f in os.listdir(mask_folder) if f.endswith(('.png', '.jpg'))]
vectores_aplanados = [aplanar_imagen(imagen) for imagen in images]
vectores_aplanados = [aplanar_imagen(mask) for mask in masks]

resized_images,resized_masks=dimension_imagenes(images,masks)
elefantonaldos=[]
for i in range(len(resized_images)):
    imagen_mascara=aplicar_mascara(resized_images[i],resized_masks[i])
    imagen_mascara.convert('RGB')
    img_prueba=np.array(imagen_mascara)
    elefantonaldos.append(img_prueba)
# Asumimos que flamencazos contiene objetos de tipo Pillow.Image
et_ele=[]
for i in range(len(elefantonaldos)):
    et_ele.append('E')


image_folder = 'objects/images/rhino'
mask_folder = 'objects/masks/rhino'

images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg'))]
masks = [os.path.join(mask_folder, f) for f in os.listdir(mask_folder) if f.endswith(('.png', '.jpg'))]
vectores_aplanados = [aplanar_imagen(imagen) for imagen in images]
vectores_aplanados = [aplanar_imagen(mask) for mask in masks]

resized_images,resized_masks=dimension_imagenes(images,masks)
rhinos=[]
for i in range(len(resized_images)):
    imagen_mascara=aplicar_mascara(resized_images[i],resized_masks[i])
    imagen_mascara.convert('RGB')
    img_prueba=np.array(imagen_mascara)
    rhinos.append(img_prueba)
et_rhin=[]
for i in range(len(rhinos)):
    et_rhin.append('R')


image_folder = 'objects/images/emu'
mask_folder = 'objects/masks/emu'

images = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg'))]
masks = [os.path.join(mask_folder, f) for f in os.listdir(mask_folder) if f.endswith(('.png', '.jpg'))]
vectores_aplanados = [aplanar_imagen(imagen) for imagen in images]
vectores_aplanados = [aplanar_imagen(mask) for mask in masks]

resized_images,resized_masks=dimension_imagenes(images,masks)
epamues=[]
for i in range(len(resized_images)):
    imagen_mascara=aplicar_mascara(resized_images[i],resized_masks[i])
    imagen_mascara.convert('RGB')
    img_prueba=np.array(imagen_mascara)
    epamues.append(img_prueba)
et_emu=[]
for i in range(len(epamues)):
    et_emu.append('O')
animales = np.concatenate([np.array(rhinos),np.array(elefantonaldos),np.array(flamencazos), np.array(epamues)])
print(len(animales))
et_animales= np.concatenate([np.array(et_rhin),np.array(et_ele),np.array(et_flam), np.array(et_emu)])
print(len(et_animales))


# CARÁCTERÍSTICAS EXTRAÍDAS:

In [ ]:


def extraer_caracteristicas_rgb(imagen):
    """
    Extrae características de color en el espacio RGB de una imagen.
    
    Parámetros:
    - imagen: La imagen de entrada (debe ser RGB, RGBA, o escala de grises).
    
    Retorna:
    - Un array con las características extraídas (valores de los canales R, G, B).
    """
    # Asegurar que la imagen es al menos 2D
    if len(imagen.shape) == 1:
        raise ValueError("La imagen parece ser 1D. Verifica el conjunto de datos.")
    
    # Convertir la imagen a formato RGB si es necesario
    if imagen.shape[-1] == 4:  # RGBA
        imagen = rgba2rgb(imagen)
    elif len(imagen.shape) == 2 or imagen.shape[-1] == 1:  # Escala de grises
        imagen = gray2rgb(imagen)
    
    # Normalizar la imagen a valores entre 0 y 1
    imagen_rgb = imagen / 255.0 if imagen.max() > 1 else imagen

    # Extraer los valores de los canales RGB
    r_channel = imagen_rgb[:, :, 0]
    g_channel = imagen_rgb[:, :, 1]
    b_channel = imagen_rgb[:, :, 2]
    
    # Calcular estadísticas para cada canal (media, desviación estándar, min, max)
    características_rgb = [
        np.mean(r_channel), np.std(r_channel), np.min(r_channel), np.max(r_channel),
        np.mean(g_channel), np.std(g_channel), np.min(g_channel), np.max(g_channel),
        np.mean(b_channel), np.std(b_channel), np.min(b_channel), np.max(b_channel)
    ]
    
    return características_rgb


def extraer_caracteristicas_hog(imagen):
    """
    Extrae las características HOG de una imagen.
    """
    # Convertir la imagen a RGB si tiene un canal alfa
    if imagen.shape[2] == 4:  # Si la imagen tiene 4 canales (RGBA)
        imagen = imagen[:, :, :3]  # Elimina el canal alfa

    # Convertir la imagen a escala de grises
    img_gray = color.rgb2gray(imagen)
    
    # Extraer las características HOG
    caracteristicas_hog, _ = hog(img_gray, block_norm='L2-Hys', pixels_per_cell=(32, 32
                                                                                ),
                                 cells_per_block=(2, 2), visualize=True)
    
    # Asegurarse de que las características HOG son un vector 1D
    return caracteristicas_hog.flatten()

def extraer_caracteristicas_color(imagen):
    """
    Extrae características de color de una imagen.
    Si la imagen no es RGB, la convierte a RGB.
    Devuelve un vector con estadísticas de color para cada canal (R, G, B).
    """
    # Convertir a RGB si es necesario
    if imagen.shape[-1] == 4:  # RGBA
        imagen = rgba2rgb(imagen)
    elif len(imagen.shape) == 2 or imagen.shape[-1] == 1:  # Escala de grises
        imagen = gray2rgb(imagen)
    
    # Separar los canales de color
    r_channel = imagen[:, :, 0]
    g_channel = imagen[:, :, 1]
    b_channel = imagen[:, :, 2]
    
    # Calcular estadísticas para cada canal
    stats = []
    for channel in [r_channel, g_channel, b_channel]:
        stats.extend([
            np.mean(channel),  # Promedio
            np.std(channel),   # Desviación estándar
            np.min(channel),   # Valor mínimo
            np.max(channel),   # Valor máximo
            np.percentile(channel, 25),  # Percentil 25
            np.percentile(channel, 50),  # Mediana
            np.percentile(channel, 75)   # Percentil 75
        ])
    
    return np.array(stats)


def extraer_caracteristicas_hsv(imagen):
    """
    Extrae características de color en el espacio HSV de una imagen.
    
    Parámetros:
    - imagen: La imagen de entrada (debe ser RGB).
    
    Retorna:
    - Un array con las características extraídas (valores de los canales H, S, V).
    """
    # Asegurarse de que la imagen esté en formato RGB
    if imagen.shape[-1] == 4:  # Si la imagen tiene un canal alfa (RGBA), eliminarlo
        imagen = imagen[:, :, :3]
    elif imagen.shape[-1] != 3:
        raise ValueError("La imagen debe tener 3 canales (RGB).")
    
    # Normalizar la imagen a valores entre 0 y 1 si es necesario
    imagen_rgb = imagen / 255.0
    
    # Convertir la imagen de RGB a HSV
    imagen_hsv = rgb2hsv(imagen_rgb)
    
    # Extraer los valores de los canales HSV
    h_channel = imagen_hsv[:, :, 0]
    s_channel = imagen_hsv[:, :, 1]
    v_channel = imagen_hsv[:, :, 2]
    
    # Calcular estadísticas para cada canal (media, desviación estándar, min, max)
    características_hsv = [
        np.mean(h_channel), np.std(h_channel), np.min(h_channel), np.max(h_channel),
        np.mean(s_channel), np.std(s_channel), np.min(s_channel), np.max(s_channel),
        np.mean(v_channel), np.std(v_channel), np.min(v_channel), np.max(v_channel)
    ]
    
    return características_hsv


def extraer_caracteristicas_textura(imagen):
    """
    Extrae características de textura utilizando la Matriz de Co-ocurrencia de Nivel de Gris (GLCM).
    
    Parámetros:
    - imagen: La imagen de entrada (puede ser RGB o en escala de grises).
    
    Retorna:
    - Un array con las características de textura (contrast, dissimilarity, homogeneity, energy, correlation, ASM).
    """
        # Asegurarse de que la imagen esté en formato RGB
    if imagen.shape[-1] == 4:  # Si la imagen tiene un canal alfa (RGBA), eliminarlo
        imagen = imagen[:, :, :3]
    # Convertir la imagen a escala de grises si es RGB
    if len(imagen.shape) == 3 and imagen.shape[-1] == 3:
        imagen = rgb2gray(imagen)
    
    # Asegurarse de que los valores estén en el rango 0-255
    imagen = (imagen * 255).astype(np.uint8)
    
    # Calcular la matriz de co-ocurrencia (distancia=1, ángulo=0°)
    glcm = graycomatrix(imagen, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    
    # Calcular propiedades de textura
    contraste = graycoprops(glcm, 'contrast').mean()
    disimilitud = graycoprops(glcm, 'dissimilarity').mean()
    homogeneidad = graycoprops(glcm, 'homogeneity').mean()
    energia = graycoprops(glcm, 'energy').mean()
    correlacion = graycoprops(glcm, 'correlation').mean()
    asm = graycoprops(glcm, 'ASM').mean()
    
    # Devolver las características como un array
    return [contraste, disimilitud, homogeneidad, energia, correlacion, asm]


# PRIMER MODELO 
Baseline, caractersiticas RGB.